In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import gc

# 1. Configurer TensorFlow pour limiter l'utilisation de la mémoire
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True) if tf.config.list_physical_devices('GPU') else None

# 2. Charger et préparer les données MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Préparation pour le CNN simple (images 28x28x1)
x_train_cnn = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test_cnn = x_test.reshape(-1, 28, 28, 1) / 255.0

# Préparation pour VGG16 (images 48x48x3)
x_train_vgg = np.repeat(x_train[..., np.newaxis], 3, axis=-1)
x_test_vgg = np.repeat(x_test[..., np.newaxis], 3, axis=-1)

# 3. Réduire le jeu d'entraînement pour VGG16 (10 000 images) AVANT resize
train_indices = np.random.choice(len(x_train_vgg), 10000, replace=False)
x_train_vgg = x_train_vgg[train_indices]
y_train_cat_vgg = to_categorical(y_train[train_indices], 10)

# Redimensionner après indexation
x_train_vgg = tf.image.resize(x_train_vgg, [48, 48]) / 255.0
x_test_vgg = tf.image.resize(x_test_vgg, [48, 48]) / 255.0

# Encodage one-hot des étiquettes pour le test
y_test_cat = to_categorical(y_test, 10)

# 4. Créer des pipelines tf.data avec petit buffer
train_dataset_cnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, to_categorical(y_train, 10))).batch(32).prefetch(1)
test_dataset_cnn = tf.data.Dataset.from_tensor_slices((x_test_cnn, y_test_cat)).batch(32).prefetch(1)

train_dataset_vgg = tf.data.Dataset.from_tensor_slices((x_train_vgg, y_train_cat_vgg)).batch(8).prefetch(1)
test_dataset_vgg = tf.data.Dataset.from_tensor_slices((x_test_vgg, y_test_cat)).batch(8).prefetch(1)

# Libérer la mémoire
del x_train, x_test, x_train_cnn, x_test_cnn, x_train_vgg, x_test_vgg, y_train, y_test, y_train_cat_vgg, y_test_cat
gc.collect()

# 5. Implémenter un CNN simple
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compiler et entraîner le CNN
cnn_model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
print("Entraînement du CNN simple...")
cnn_model.fit(train_dataset_cnn, epochs=5, validation_data=test_dataset_cnn)

# Libérer la mémoire après le CNN
del cnn_model
gc.collect()

# 6. Transfert d'apprentissage avec VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
output = Dense(10, activation='softmax')(x)
vgg_model = Model(inputs=base_model.input, outputs=output)

# Compiler et entraîner VGG16
vgg_model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
print("Entraînement du modèle VGG16 (Transfer Learning)...")
vgg_model.fit(train_dataset_vgg, epochs=3, validation_data=test_dataset_vgg)

Entraînement du CNN simple...
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 63s 32ms/step - accuracy: 0.9149 - loss: 0.2858 - val_accuracy: 0.9749 - val_loss: 0.0701
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 33ms/step - accuracy: 0.9842 - loss: 0.0514 - val_accuracy: 0.9868 - val_loss: 0.0424
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 66s 35ms/step - accuracy: 0.9898 - loss: 0.0332 - val_accuracy: 0.9888 - val_loss: 0.0335
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 58s 31ms/step - accuracy: 0.9931 - loss: 0.0212 - val_accuracy: 0.9898 - val_loss: 0.0322
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 32ms/step - accuracy: 0.9956 - loss: 0.0139 - val_accuracy: 0.9896 - val_loss: 0.0378
Entraînement du modèle VGG16 (Transfer Learning)...
Epoch 1/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 576s 459ms/step - accuracy: 0.4401 - loss: 1.8854 - val_accuracy: 0.7781 - val_loss: 1.0484
Epoch 2/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 569s 456ms/step - accuracy: 0.8153 - loss: 0.9147 - val_accuracy: 0.8447 - val_loss: 0.6839
Epoch 3/3
1250/1250 ━━━━━━━━━━━━━━━━━

In [6]:
# 6. Transfert d'apprentissage avec ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
output = Dense(10, activation='softmax')(x)
resnet_model = Model(inputs=base_model.input, outputs=output)

# Compiler et entraîner ResNet50
resnet_model.compile(optimizer=Adam(learning_rate=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
print("Entraînement du modèle ResNet50 (Transfer Learning)...")
resnet_model.fit(train_dataset_vgg, epochs=3, validation_data=test_dataset_vgg)

Entraînement du modèle ResNet50 (Transfer Learning)...
Epoch 1/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 261s 202ms/step - accuracy: 0.4773 - loss: 1.7362 - val_accuracy: 0.7518 - val_loss: 0.9500
Epoch 2/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 248s 198ms/step - accuracy: 0.7669 - loss: 0.8911 - val_accuracy: 0.8199 - val_loss: 0.6732
Epoch 3/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 254s 203ms/step - accuracy: 0.8190 - loss: 0.6715 - val_accuracy: 0.8516 - val_loss: 0.5449
